<a href="https://colab.research.google.com/github/ccasanoval/RLtests/blob/master/LanderV0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lunar Lander v0 for HuggingFace

Install required libs

In [1]:
!pip install stable-baselines3==2.0.0a5
!pip install swig
!pip install gymnasium[box2d]
!pip install huggingface_sb3

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install pyvirtualdisplay

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb'

Import required libs

In [2]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import (
    notebook_login,
)  # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

Play around with the env

In [3]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
    # Take a random action
    action = env.action_space.sample()
    print("Action taken:", action)

    # Do this action in the environment and get
    # next_state, reward, terminated, truncated and info
    observation, reward, terminated, truncated, info = env.step(action)

    # If the game is terminated (in our case we land, crashed) or truncated (timeout)
    if terminated or truncated:
        # Reset the environment
        print("Environment is reset")
        observation, info = env.reset()

env.close()

Action taken: 2
Action taken: 3
Action taken: 0
Action taken: 3
Action taken: 0
Action taken: 1
Action taken: 0
Action taken: 3
Action taken: 2
Action taken: 0
Action taken: 1
Action taken: 3
Action taken: 1
Action taken: 2
Action taken: 1
Action taken: 2
Action taken: 2
Action taken: 1
Action taken: 3
Action taken: 3


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v2")
env.reset()

print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())  # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())  # Take a random action

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [ 43.105686   -59.092484     3.3241234    0.46600187   0.47528875
  -1.4407223    0.61227465   0.96833867]

 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 2


Create Agent Algo and Train it

In [5]:
#Vectorized env
env = make_vec_env("LunarLander-v2", n_envs=16)

#env = gym.make('LunarLander-v2')

# Instantiate the agent
#model = PPO('MlpPolicy', env, verbose=1)
model = PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1,
)

# Train the agent
model.learn(total_timesteps=int(1e6))
model_name = "ppo-LunarLander-v2"
model.save(model_name)



Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 96       |
|    ep_rew_mean     | -209     |
| time/              |          |
|    fps             | 2897     |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 85.3        |
|    ep_rew_mean          | -139        |
| time/                   |             |
|    fps                  | 2071        |
|    iterations           | 2           |
|    time_elapsed         | 15          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.009165821 |
|    clip_fraction        | 0.0663      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.00209     |
|    learnin

Evaluate the Agent

In [6]:
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=257.32 +/- 22.711605826300385
